# ArcPy Review and Introduction

<ul>
    <li>Tuple Unpacking</li>
    <li>Interacting with Feature Layers and Feature Classes</li>
    <li>Interfacing with the map and map pane</li>
    <li>Running geoprocessing tools</li>
</ul>

In [ ]:
import os
import arcpy

Get current project default GDB

In [ ]:
arcpy.env.workspace

Typically I utilize the default GDB of a project, <br>
but when I need to save the outputs or intermediate files to a specific GDB, <br>
I will change the workspace to that GDB:

In [ ]:
#get path to current directory
current_folder = os.getcwd()

#create a path to the other example gdb within the directory
example_gdb = os.path.join(current_folder, 'Example.gdb')

#reset the current workspace to the example gdb path
arcpy.env.workspace = example_gdb

In [ ]:
example_gdb

In [ ]:
arcpy.env.workspace

For now, we will continue to use the default project GDB, <br>
so in the below cell, we will switch it back to the original:

In [ ]:
#create a path to the default GDB
session2_gdb = os.path.join(current_folder, 'Session2_ArcPyCursors.gdb')

#reset the current workspace to the example gdb path
arcpy.env.workspace = session2_gdb

In [ ]:
arcpy.env.workspace

There are other important methods we can utilize in the env class <br>
The most common that I use are the following:

In [ ]:
arcpy.env.addOutputsToMap = True
arcpy.env.overwriteOutput = True

There are other environmental settings within arcpy, you can find a list <br>
of them within the following link

https://pro.arcgis.com/en/pro-app/latest/arcpy/classes/env.htm

Now lets find some data and add it to the projects GDB!
We can use OS and arcpy to help us with this:

In [ ]:
current_folder = os.getcwd()

# path to data folder
data_folder = os.path.join(current_folder, 'Data')

#list all the files in the data_folder
os.listdir(data_folder)

In [ ]:
data_folder

We could use os and look for any shapefiles within the folder, <br>
recreate the paths, and then add them to the GDB.
<br> Instead! we can actually use an arcpy function to help us with this!

<b> ArcPy Walk </b> <br>


In [ ]:
walk = arcpy.da.Walk(data_folder)

In [ ]:
walk

In [ ]:
for f in walk:
    print(f)

ArcPy Walk returns a tuple, which can be thought of as an iterable object (sort of like a list! but immutable). <br>
We need to iterate over the tuple to get after the values in the iterable.

<br>
<br>
Notice there are three positions in the tuple:
<br>
<b>The Folder Path : The folders within the input folder : any geo data</b>

<br>
<br>
We can work with this data by using Tuple Unpacking, which we can use to seperate out each value

In [ ]:
# tuple unpacking example

sample_tuple = ('eric', 30, 'samson')

# first_name, age, last_name = sample_tuple

# f'{first_name}_{last_name} is {age} years old'

In [ ]:
walk = arcpy.da.Walk(data_folder)

for dirpath, dirnames, filenames in walk:
    print(dirpath)
    print(dirnames)
    print(filenames)

In [ ]:
walk = arcpy.da.Walk(data_folder)

for dirpath, dirnames, filenames in walk:
    for filename in filenames:
        file_path = os.path.join(dirpath, filename)
        print(file_path)

<b> Before continuing, it's good to note that there are additional <br>
    options that you can pass into ArcPy Walk</b>
    
https://pro.arcgis.com/en/pro-app/latest/arcpy/data-access/walk.htm

In [ ]:
walk = arcpy.da.Walk(data_folder, datatype="FeatureClass", type="Polygon")

In [ ]:
for f in walk:
    print(f)

For use case, we only need to pass in the data folder path:

In [ ]:
walk = arcpy.da.Walk(data_folder)

data_list = []
for dirpath, dirnames, filenames in walk:
    for filename in filenames:
        file_path = os.path.join(dirpath, filename)
        #print(file_path)
        data_list.append(file_path)

In [ ]:
data_list

<b> Great! Now we have a list of the shapefile paths, lets copy them into our GDB </b>

<br>
To do this, we will use the copy features tool.

https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/copy-features.htm

In [ ]:
test = os.path.basename('C:\\Users\\eric.samson\\Documents\\Python\\BayGeo_IntermediateClass\\Workshop_2.0\\Workshop2\\Workshop2\\Data\\sf_neighborhoods_project.shp')

In [ ]:
os.path.splitext(test)[0]

In [ ]:
data_list

In [ ]:
arcpy.env.workspace

In [ ]:
#loop through our shapefiles
for shapefile_path in data_list:
    # recreate the name of the shapefile, without the extension
    basename = os.path.basename(shapefile_path)
    new_fc_name = os.path.splitext(basename)[0]

    # Recreate the output path using the new name
    out_featureclass = os.path.join(arcpy.env.workspace, 
                                    new_fc_name)

    # copy the shapefile to the new location
    arcpy.management.CopyFeatures(shapefile_path, 
                                  out_featureclass)

<b> We should now have our data within our Map Pane, which means they are now within our default GDB! </b>

<b> Lets review this data and interact with it in python</b>
    
<br>
Now that the data is within the map pane, it is now much easier for us to interact with it within our ArcGIS Notebook

<b> We can do geoprocessing tasks using our layers in the map pane! </b>

<br>
<br>
Let's try to merge the point files!
<br>

https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/merge.htm

In [ ]:
arcpy.management.Merge("sf_starbucks;sf_peets", 
                       "sf_coffee")

After Running the above cell, we should get a new feature layer in our map pane called sf_coffee!

<br>
What if we wanted to do a buffer? An Erase Overlay?

<br>
All possible in python when the layers are in the map pane!

If you don't want to save the output file permanently, 
<br>
<b>you can use ArcGIS Pros temporary memory!</b>
<br>

This is very handy when stringing together arcpy functions!

In [ ]:
sf_coffee_buffer = arcpy.analysis.Buffer("sf_coffee", 
                                          r"memory\sf_coffee_buffer", 
                                          "50 Meters")

arcpy.analysis.Erase("sf_neighborhoods_project", 
                    sf_coffee_buffer,
                    r"memory\sf_coffee_nhood_clip")

Let's add some neighborhood information to our coffee points layer using a spatial join!

https://pro.arcgis.com/en/pro-app/2.9/tool-reference/analysis/spatial-join.htm

In [ ]:
arcpy.SpatialJoin_analysis("sf_coffee", 
                           "sf_neighborhoods_project", 
                          "sf_coffee_sp")

In [ ]:
fields_to_delete = ["Join_Count", "TARGET_FID"]
arcpy.DeleteField_management("sf_coffee_sp", fields_to_delete)

In [ ]:
# re-run everything again for demonstration